# 医学AI多模态Transformer分割入门教程（可运行）

本教程面向**完全不会代码的新手**，一步一步演示如何用**Transformer**做**医学图像分割**，并融合**多模态数据**（图像 + 文本/结构化信息）。

> 为了确保**任何人都能跑通**，我们使用**可控的合成数据**进行演示。
> 真实项目中只需要把数据读取部分替换成你的医学影像即可。

你将学到：
1. 如何准备多模态数据（图像 + 文本）
2. 如何构建一个简化的Transformer分割模型
3. 如何训练、评估并保存模型


## 0. 环境准备

如果你还没有安装依赖，可以运行下面的命令：

```bash
pip install torch torchvision numpy matplotlib
```

> 提示：如果你用的是GPU环境，可安装支持CUDA的PyTorch版本。


In [ ]:
# 1. 导入依赖
import math
import random
from dataclasses import dataclass

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt


In [ ]:
# 2. 固定随机种子，保证每次结果可复现
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)


## 3. 准备“多模态”数据（合成示例）

我们构造一个可运行的小数据集：
- **图像**：64x64 的灰度图（模拟CT/MRI切片）
- **文本模态**：简短的病人描述（例如“肿瘤大小大/小”）
- **标签**：每张图像对应一个分割掩码（白色区域代表病灶）

> 真实项目中，你需要把这里替换为读取DICOM/NIfTI数据、病人报告等。


In [ ]:
# 3.1 生成合成分割数据
# 我们用简单的几何图形模拟“病灶”区域

def make_circle_mask(size=64, radius=10, center=None):
    mask = np.zeros((size, size), dtype=np.float32)
    if center is None:
        center = (size // 2, size // 2)
    yy, xx = np.ogrid[:size, :size]
    cy, cx = center
    dist = (yy - cy) ** 2 + (xx - cx) ** 2
    mask[dist <= radius ** 2] = 1.0
    return mask


def generate_sample(size=64):
    # 随机生成“病灶”大小和位置
    radius = np.random.randint(6, 14)
    center = (np.random.randint(16, 48), np.random.randint(16, 48))

    mask = make_circle_mask(size=size, radius=radius, center=center)

    # 图像 = 噪声 + 病灶区域增强
    image = np.random.randn(size, size).astype(np.float32) * 0.2
    image += mask * np.random.uniform(0.8, 1.2)

    # 文本模态: 简单描述（可扩展为病历文本）
    if radius >= 10:
        text = "tumor is large"
    else:
        text = "tumor is small"

    return image, mask, text


In [ ]:
# 3.2 构建PyTorch数据集

# 简单的词表（用于把文本转换成数字）
vocab = {
    "tumor": 0,
    "is": 1,
    "large": 2,
    "small": 3,
    "<pad>": 4,
}


def text_to_ids(text, max_len=3):
    tokens = text.split()
    ids = [vocab.get(t, vocab["<pad>"]) for t in tokens]
    # 补齐长度
    if len(ids) < max_len:
        ids += [vocab["<pad>"]] * (max_len - len(ids))
    return ids[:max_len]


class SyntheticMedicalDataset(Dataset):
    def __init__(self, num_samples=200, size=64):
        self.samples = [generate_sample(size) for _ in range(num_samples)]

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        image, mask, text = self.samples[idx]
        image = torch.tensor(image).unsqueeze(0)  # [1, H, W]
        mask = torch.tensor(mask).unsqueeze(0)    # [1, H, W]
        text_ids = torch.tensor(text_to_ids(text))
        return image, mask, text_ids


In [ ]:
# 3.3 创建DataLoader
train_dataset = SyntheticMedicalDataset(num_samples=200)
val_dataset = SyntheticMedicalDataset(num_samples=50)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8)


In [ ]:
# 3.4 可视化一个样本
image, mask, text_ids = train_dataset[0]

plt.figure(figsize=(6, 3))
plt.subplot(1, 2, 1)
plt.title("Image")
plt.imshow(image.squeeze().numpy(), cmap="gray")

plt.subplot(1, 2, 2)
plt.title("Mask")
plt.imshow(mask.squeeze().numpy(), cmap="gray")
plt.show()

print("Text ids:", text_ids)


## 4. 构建Transformer分割模型（简化版）

我们实现一个**极简的多模态Transformer**：
- 图像被分成小块（patch）
- 每个patch转成token后送入Transformer编码器
- 文本也转成embedding
- 将图像特征与文本特征融合
- 最后输出分割mask

> 为了教程简洁，我们只演示核心思想，代码可运行。


In [ ]:
# 4.1 模型定义

class PatchEmbedding(nn.Module):
    def __init__(self, img_size=64, patch_size=8, in_chans=1, embed_dim=64):
        super().__init__()
        self.img_size = img_size
        self.patch_size = patch_size
        self.n_patches = (img_size // patch_size) ** 2
        self.proj = nn.Conv2d(in_chans, embed_dim, kernel_size=patch_size, stride=patch_size)

    def forward(self, x):
        # x: [B, C, H, W]
        x = self.proj(x)  # [B, embed_dim, H/ps, W/ps]
        x = x.flatten(2).transpose(1, 2)  # [B, N, embed_dim]
        return x


class MultiModalTransformerSeg(nn.Module):
    def __init__(self, img_size=64, patch_size=8, vocab_size=5, embed_dim=64, num_layers=2, num_heads=4):
        super().__init__()
        self.patch_embed = PatchEmbedding(img_size, patch_size, 1, embed_dim)

        self.text_embed = nn.Embedding(vocab_size, embed_dim)

        encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads, batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.fusion_fc = nn.Linear(embed_dim * 2, embed_dim)

        # 将token还原为feature map
        self.up_proj = nn.Linear(embed_dim, patch_size * patch_size)
        self.patch_size = patch_size
        self.img_size = img_size

    def forward(self, img, text_ids):
        # 图像patch tokens
        img_tokens = self.patch_embed(img)  # [B, N, D]

        # 文本token -> 平均作为文本特征
        text_embeds = self.text_embed(text_ids)  # [B, T, D]
        text_feat = text_embeds.mean(dim=1, keepdim=True)  # [B, 1, D]

        # 融合：把文本特征扩展到所有patch
        text_feat_expanded = text_feat.repeat(1, img_tokens.size(1), 1)
        fused = torch.cat([img_tokens, text_feat_expanded], dim=-1)
        fused = self.fusion_fc(fused)

        # Transformer编码
        encoded = self.transformer(fused)  # [B, N, D]

        # 解码成mask
        patch_logits = self.up_proj(encoded)  # [B, N, patch_size*patch_size]

        # 还原到整图大小
        B, N, _ = patch_logits.shape
        patches = patch_logits.view(B, N, self.patch_size, self.patch_size)
        grid_size = self.img_size // self.patch_size
        patches = patches.view(B, grid_size, grid_size, self.patch_size, self.patch_size)
        mask = patches.permute(0, 1, 3, 2, 4).reshape(B, 1, self.img_size, self.img_size)
        return mask


## 5. 训练与评估

我们使用：
- **Loss**：二元交叉熵（BCE）
- **指标**：Dice系数（越高越好）


In [ ]:
# 5.1 定义Loss与评估指标

def dice_score(pred, target, eps=1e-6):
    pred = (pred > 0.5).float()
    intersection = (pred * target).sum()
    union = pred.sum() + target.sum()
    return (2 * intersection + eps) / (union + eps)


model = MultiModalTransformerSeg()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.BCEWithLogitsLoss()


In [ ]:
# 5.2 训练循环

for epoch in range(3):
    model.train()
    total_loss = 0.0

    for images, masks, text_ids in train_loader:
        logits = model(images, text_ids)
        loss = criterion(logits, masks)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)

    # 验证
    model.eval()
    dice_scores = []
    with torch.no_grad():
        for images, masks, text_ids in val_loader:
            logits = model(images, text_ids)
            probs = torch.sigmoid(logits)
            dice = dice_score(probs, masks)
            dice_scores.append(dice.item())

    print(f"Epoch {epoch+1}: loss={avg_loss:.4f}, dice={np.mean(dice_scores):.4f}")


## 6. 推理与可视化结果

训练完成后，我们可在一张样本上进行预测并可视化。


In [ ]:
# 6.1 推理示例
model.eval()
image, mask, text_ids = val_dataset[0]

with torch.no_grad():
    logits = model(image.unsqueeze(0), text_ids.unsqueeze(0))
    pred = torch.sigmoid(logits).squeeze().numpy()

plt.figure(figsize=(9, 3))
plt.subplot(1, 3, 1)
plt.title("Image")
plt.imshow(image.squeeze().numpy(), cmap="gray")

plt.subplot(1, 3, 2)
plt.title("Ground Truth")
plt.imshow(mask.squeeze().numpy(), cmap="gray")

plt.subplot(1, 3, 3)
plt.title("Prediction")
plt.imshow(pred, cmap="gray")
plt.show()


## 7. 保存模型

你可以保存训练好的模型参数，之后加载继续使用。


In [ ]:
# 7.1 保存模型

torch.save(model.state_dict(), "multimodal_transformer_seg.pth")
print("模型已保存")


## 8. 下一步建议（真实医学场景）

当你准备处理真实数据时，可以逐步替换：
1. **数据读取**：使用 `pydicom`/`nibabel` 读取DICOM或NIfTI
2. **文本模态**：使用真实病历文本或结构化信息
3. **模型升级**：替换为更强的Transformer（如Swin-UNet、ViT-UNet）
4. **训练优化**：更多数据、数据增强、学习率调度等

如果需要，我可以帮你把这个教程升级成“真实医学影像数据版本”。
